## Группы и объект сопоставления

Каждая пара круглых скобок в регулярном выражении создает группу. Группы нумеруются по порядку открывающих скобок:


In [9]:
import re

expr = r'(\d+):([a-z]+)'  # создали две группы, для последовательности цифр, для последовательности букв

m = re.fullmatch(expr, '123:abc')
print(m)
print(m.group(1))  # получить группу по номеру
print(m.group(2))  # получить группу по номеру
print(m.groups())  # тьюпл из всех групп
print(m.group()) # группа без номера — то, с чем сопоставилось выражение целиком
print(m.pos)  # позиция в исходной строке, где было найдено сопоставление
print(m.start())  # начало и конец сопоставления по индексу
print(m.end())

print(m.start(1))  # начало и конец сопоставления по индексу группы 1
print(m.end(1))
print(m.span())  # начало и конец в виде тьюпла

<re.Match object; span=(0, 7), match='123:abc'>
123
abc
('123', 'abc')
123:abc
0
0
7
0
3
(0, 7)


Аналогичный объект получаем, если делаем поиск:


In [11]:
for m in re.finditer(expr, "123:abc ******  456:xyzt ***"):
    print(f'{m.group(1)} и {m.group(2)}')


123 и abc
456 и xyzt


Про сложные регулярные выражения. Не используйте сложные регулярные выражения, если приходится — решите задачу иначе. "Если у вас была проблема, и вы решили ее с помощью регулярных выражений, то теперь у вас две проблемы". В общем, пользуйтесь только простыми.

Допустим, мы хотим искать в тексте упоминание моментов времени. Вид ЧЧ:ММ (часы и минуты через двоеточие). Мы знаем, что не бывает `24:89`, потому что часов не больше 23, минут не больше 59:


In [12]:
time_expr = r'([01][0-9]|2[0-3]):([0-5][0-9])'
# r'(00|01|02|03|04|...|23)'
text = "Первое упоминание времени 12:57, неправильное упоминание 44:87 и еще 05:00"

for m in re.finditer(time_expr, text):
    print(f'Найдено время {m.group(1)} часов и {m.group(2)} минут')


Найдено время 12 часов и 57 минут
Найдено время 05 часов и 00 минут


Не найдено 44:87!
Но это сложно, надо делать иначе. Регулярные выражения должны искать строки с правильным синтаксисом, но лучше не должны проверять семантику (смысл).


In [13]:
time_expr = r'(\d\d):(\d\d)'
text = "Первое упоминание времени 12:57, неправильное упоминание 44:87 и еще 05:00"

for m in re.finditer(time_expr, text):
    hours = int(m.group(1))
    minutes = int(m.group(2))
    # проверяем смысл найденного
    if 0 <= hours <= 23 and 0 <= minutes <= 59:
        print(f'Найдено время {hours} часов и {minutes} минут')

Найдено время 12 часов и 57 минут
Найдено время 5 часов и 0 минут


# Формат данных JSON

Часто компьютер обрабатывает структурированные данные, и хранить такие данные в виде обычного текста — неудобно.
Превращать данные в текст необходимо, чтобы обменивать ими с другими программами. Другие программы могут быть написаны на другом языке, запущены на другом компьютере, но если формат представления данных знаком обеим программам, они обе смогут им воспользоваться.

Пример неудобного хранения данных:

```
1 Посов Илья курс 1
2 Иванов Иван курс 2
3 Иванов Сергей курс 1
```

Чтобы это прочитать нужен код наподобие:

```
with open(...) as f:
    for line in f.readlines():
        words = line.split()  # разделить по пробелам: ['1', 'Посов', 'Илья', 'курс', '1']
        surname = words[1]
        name = words[2]
        course = int(words[4])
```

Бывает, что данные не чистые, где-то пропущена фамилия, курс указан не числом и т.д. Код для разбора всех этих случаев будет слишком длинный.

Лучше подобные данные хранить иначе. Стандарт JSON = JavaScript Object Notation, данные в формате json — это корректный JS-код (и часто еще и корректный Python):

```
[
    {
        "surname": "Посов",
        "name": "Илья",
        "course": 1
    },
    {
        "surname": "Иванов",
        "name": "Иван",
        "course": 2
    },
    {
        "surname": "Иванов",
        "name": "Сергей",
        "course": 1
    }
]
```

Пусть есть текст в таком формате, его можно преобразовать в Python-данные с помощью возможностей встроенного пакета `json`:

In [17]:
students_as_json = """
[
    {
        "surname": "Посов",
        "name": "Илья",
        "course": 1
    },
    {
        "surname": "Иванов",
        "name": "Иван",
        "course": 2
    },
    {
        "surname": "Иванов",
        "name": "Сергей",
        "course": 1
    }
]
"""

import json

students = json.loads(students_as_json)  # получаем значение в Python, здесь это список
for student in students:  # список словарей
    print(student)  # каждый student это словарь
    print(f'Имя {student["name"]} Фамилия {student["surname"]} Курс {student["course"]}')

{'surname': 'Посов', 'name': 'Илья', 'course': 1}
Имя Илья Фамилия Посов Курс 1
{'surname': 'Иванов', 'name': 'Иван', 'course': 2}
Имя Иван Фамилия Иванов Курс 2
{'surname': 'Иванов', 'name': 'Сергей', 'course': 1}
Имя Сергей Фамилия Иванов Курс 1


Аналогично можно преобразовывать значения из Python в json-текст


In [21]:
marks = [3, 4, 3, 3, 3, 5, 2]
data = {"student": {"name": "Ilya", "surname": "Posov"}, "marks": marks}

#превратить это в json-строку:
data_as_json = json.dumps(data)

print(data_as_json)

{"student": {"name": "Ilya", "surname": "Posov"}, "marks": [3, 4, 3, 3, 3, 5, 2]}


Аналогичные методы `dump` и `load` сохраняют и читают из файла. Еще в пакете `json` есть классы JsonEncoder, JsonDecoder, работой которых можно управлять. Например, при записи json можно не ставить лишние пробелы, чтобы результат был как можно короче. Можно наоборот ставить много переводов строк для улучшения читаемости.
